#딥러닝 final project

2022320009 이수현

## 데이터 전처리 및 데이터 확인

* 파일을 살펴보다가 중복을 발견
* SampleMoviePosters와 samplemovieposters가 중복 (전체 파일 중복) -> samplemovieposters 삭제

[데이터확인]

* 총 영화 포스터의 개수 : SampleMoviePosters안에 있는 jpg파일 개수 확인
* 영화 장르가 적혀있는 csv 파일 확인

# 데이터 준비

In [1]:
# kaggle.json 파일 올려야함
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
#kaggle에서 데이터 가져오기
!kaggle datasets download -d neha1703/movie-genre-from-its-poster

!unzip movie-genre-from-its-poster.zip -d ./movie_genre_data

Dataset URL: https://www.kaggle.com/datasets/neha1703/movie-genre-from-its-poster
License(s): unknown
100% 35.9M/35.9M [00:02<00:00, 22.6MB/s]
100% 35.9M/35.9M [00:02<00:00, 13.8MB/s]
Archive:  movie-genre-from-its-poster.zip
  inflating: ./movie_genre_data/MovieGenre.csv  
  inflating: ./movie_genre_data/SampleMoviePosters/SampleMoviePosters/10040.jpg  
  inflating: ./movie_genre_data/SampleMoviePosters/SampleMoviePosters/10057.jpg  
  inflating: ./movie_genre_data/SampleMoviePosters/SampleMoviePosters/10071.jpg  
  inflating: ./movie_genre_data/SampleMoviePosters/SampleMoviePosters/10155.jpg  
  inflating: ./movie_genre_data/SampleMoviePosters/SampleMoviePosters/10195.jpg  
  inflating: ./movie_genre_data/SampleMoviePosters/SampleMoviePosters/10208.jpg  
  inflating: ./movie_genre_data/SampleMoviePosters/SampleMoviePosters/10247.jpg  
  inflating: ./movie_genre_data/SampleMoviePosters/SampleMoviePosters/10267.jpg  
  inflating: ./movie_genre_data/SampleMoviePosters/SampleMoviePosters

In [3]:
import os

#파일 확인!
dataset_path = "./movie_genre_data"
print(os.listdir(dataset_path))

['SampleMoviePosters', 'samplemovieposters', 'MovieGenre.csv']


## 데이터 전처리 및 데이터 확인

* 파일을 살펴보다가 중복을 발견
* SampleMoviePosters와 samplemovieposters가 중복 (전체 파일 중복) -> samplemovieposters 삭제

[데이터확인]

* 총 영화 포스터의 개수 : SampleMoviePosters안에 있는 jpg파일 개수 확인
* 영화 장르가 적혀있는 csv 파일 확인

In [ ]:
import shutil
import os

#samplemovieposters 파일 삭제
path_to_duplicate = "./movie_genre_data/samplemovieposters"
if os.path.exists(path_to_duplicate):
    shutil.rmtree(path_to_duplicate)
    print(f"{path_to_duplicate} 삭제 완료.")
else:
    print("중복 폴더가 없습니다.")

./movie_genre_data/samplemovieposters 삭제 완료.


In [ ]:
#파일 안에 영화 포스터 개수 세기
movieposter_path = "./movie_genre_data/SampleMoviePosters/SampleMoviePosters"

poster_files = [file for file in os.listdir(movieposter_path) if file.endswith('.jpg')]
poster_count = len(poster_files)

print(f"총 JPG 파일 개수: {poster_count}") #997개

총 JPG 파일 개수: 997


In [ ]:
#장르csv파일 확인
import pandas as pd
genrecsv_path = "./movie_genre_data/MovieGenre.csv"
data = pd.read_csv(genrecsv_path, encoding="latin1")

print(data.columns) #Index(['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster'], dtype='object')

Index(['imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster'], dtype='object')


MovieGenre.csv 파일 확인
* column은 'imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster' 로 구성
* 수동 확인 끝에 영화 포스터 이미지의 제목 = imdbId임을 확인
ex. 10057.jpg는 imdbId가 10057의 영화 포스터
* 필요한 정보는 imdbId, Genre.
* 두 정보를 제외하고 모두 삭제

In [ ]:
#imdbId, Genre만 선택

genredata = data[["imdbId", "Genre"]]
print(genredata.head())

   imdbId                       Genre
0  114709  Animation|Adventure|Comedy
1  113497     Action|Adventure|Family
2  113228              Comedy|Romance
3  114885        Comedy|Drama|Romance
4  113041       Comedy|Family|Romance


* 한 영화에 여러 장르가 중복인 것을 확인
* 장르 정보 알아보기

In [ ]:
from collections import Counter

# 'Genre' 열의 값을 '|' 기준으로 분리하고 평탄화
all_genres = [genre for genres in genredata["Genre"].dropna() for genre in genres.split('|')]

genre_counts = Counter(all_genres)
genre_list = list(genre_counts.keys())
genre_total_count = len(genre_list)

print(f"장르 개수: {genre_total_count}")
print(f"장르 목록: {genre_list}")
print("\n장르별 영화 개수:")
for genre, count in genre_counts.items():
    print(f"{genre}: {count}")

장르 개수: 28
장르 목록: ['Animation', 'Adventure', 'Comedy', 'Action', 'Family', 'Romance', 'Drama', 'Crime', 'Thriller', 'Fantasy', 'Horror', 'Biography', 'History', 'Mystery', 'Sci-Fi', 'War', 'Sport', 'Music', 'Documentary', 'Musical', 'Western', 'Short', 'Film-Noir', 'Talk-Show', 'News', 'Adult', 'Reality-TV', 'Game-Show']

장르별 영화 개수:
Animation: 1750
Adventure: 3829
Comedy: 12682
Action: 5331
Family: 2100
Romance: 6224
Drama: 20053
Crime: 5270
Thriller: 4816
Fantasy: 2012
Horror: 3990
Biography: 1999
History: 1426
Mystery: 2395
Sci-Fi: 2020
War: 1173
Sport: 707
Music: 1292
Documentary: 3882
Musical: 845
Western: 853
Short: 1066
Film-Noir: 403
Talk-Show: 7
News: 83
Adult: 14
Reality-TV: 2
Game-Show: 1


[이미지 데이터 전처리]
* 이미지 크기 조정 : CLIP 모델은 보통 224X224 크기
* 정규화 : 이미지 픽셀 값을 [0.1]
* 이미지와 genre의 라벨링

In [ ]:
import os
import pandas as pd
from PIL import Image
from torchvision import transforms
from tqdm import tqdm  # 진행 바 표시

# 이미지 전처리 파이프라인
image_transform = transforms.Compose([
    transforms.Resize((224, 224)), #보통 clip는 224X224
    transforms.ToTensor(), #[0,1]의 텐서로 바꾸기
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), #정규화
])

# 전처리된 이미지 데이터와 imdbId 저장
processed_images = []
image_ids = [] #(ID, imdbId)

# 전처리 수행
for filename in tqdm(os.listdir(movieposter_path), desc="Processing Images"):
    if filename.endswith('.jpg'):
        image_path = os.path.join(movieposter_path, filename)

        # 이미지 불러오기
        try:
            image = Image.open(image_path).convert("RGB")
            processed_image = image_transform(image)
            processed_images.append(processed_image)

            # imdbId 저장 (파일 이름에서 확장자 제거)
            image_id = os.path.splitext(filename)[0]
            image_ids.append(image_id)
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# 결과 확인
print(f"전처리된 이미지 개수: {len(processed_images)}")

#image_ids[i] == processed_images[i]

Processing Images: 100%|██████████| 997/997 [00:03<00:00, 251.85it/s]

전처리된 이미지 개수: 997


In [ ]:
index = 0  # 첫 번째 이미지
print("Image Tensor:", processed_images[index])  # 첫 번째 이미지의 텐서 출력
print("IMDB ID:", image_ids[index])  # 첫 번째 이미지의 imdbId 출력

Image Tensor: tensor([[[ 0.5608,  0.4118,  0.3647,  ...,  0.6000,  0.5922,  0.5843],
         [ 0.5765,  0.4353,  0.3961,  ...,  0.4118,  0.4039,  0.3961],
         [ 0.6157,  0.4745,  0.4275,  ...,  0.4353,  0.4353,  0.4275],
         ...,
         [ 0.6706,  0.5686,  0.4039,  ...,  0.1686,  0.4980,  0.5059],
         [ 0.6863,  0.5608,  0.3804,  ...,  0.1765,  0.4902,  0.4745],
         [ 0.6941,  0.5686,  0.3804,  ...,  0.1765,  0.4745,  0.4588]],

        [[ 0.5137,  0.3647,  0.3176,  ...,  0.5294,  0.5216,  0.5137],
         [ 0.5294,  0.3882,  0.3412,  ...,  0.3412,  0.3333,  0.3255],
         [ 0.5608,  0.4196,  0.3647,  ...,  0.3490,  0.3490,  0.3490],
         ...,
         [ 0.6235,  0.5216,  0.3647,  ...,  0.0510,  0.3647,  0.3725],
         [ 0.6392,  0.5137,  0.3412,  ...,  0.0588,  0.3569,  0.3412],
         [ 0.6471,  0.5216,  0.3412,  ...,  0.0588,  0.3412,  0.3255]],

        [[ 0.3255,  0.1765,  0.1137,  ...,  0.2706,  0.2627,  0.2549],
         [ 0.3176,  0.1765,  0.

위 과정에서의 우려 : 이미지 크기 문제 : 원본 이미지가 너무 작아서 리사이즈할 때 왜곡이 발생할 수 있음!
심지어 원래 영화포스터는 정사각형이 아니라 더 걱정,,,

In [ ]:
# 이미지 파일 확인
image_filenames = os.listdir(movieposter_path)
image_ids = [os.path.splitext(filename)[0] for filename in image_filenames if filename.endswith('.jpg')]

# 이미지 파일에 해당하는 imdbId와 장르 필터링
matched_data = []
for image_id in image_ids:
    if image_id in genredata["imdbId"].astype(str).values:  # imdbId와 매칭
        genre = genredata.loc[genredata["imdbId"].astype(str) == image_id, "Genre"].values[0]

        # 파일 이름에서 확장자 제거하고 저장
        image_filename = os.path.splitext(f"{image_id}.jpg")[0]  # 확장자 제거
        matched_data.append((image_filename, genre))  # 확장자 제거된 이름과 장르 저장

# 결과 확인
print(f"매칭된 데이터 개수: {len(matched_data)}")
print(f"예시: {matched_data[:5]}")

매칭된 데이터 개수: 997
예시: [('5529', 'Crime|Drama|Mystery'), ('14497', 'Comedy|Drama|Romance'), ('3952', 'Crime|Drama'), ('25132', 'Comedy|Drama|Romance'), ('11130', 'Drama|Horror|Sci-Fi')]


* 위의 두 코드의 결과를 통합하여 index, imdbId, 이미지 텐서, genre를 포함한 데이터셋 만들기

In [ ]:
final_data = []
for idx, (processed_image, image_id) in enumerate(zip(processed_images, image_ids)):
    if image_id in genredata["imdbId"].astype(str).values:  # imdbId와 매칭
        genre = genredata.loc[genredata["imdbId"].astype(str) == image_id, "Genre"].values[0]
        final_data.append((idx, image_id, processed_image, genre))

# 결과 확인
print(f"최종 데이터셋 크기: {len(final_data)}")
print(f"예시: {final_data[:5]}")

최종 데이터셋 크기: 997
예시: [(0, '5529', tensor([[[ 0.5608,  0.4118,  0.3647,  ...,  0.6000,  0.5922,  0.5843],
         [ 0.5765,  0.4353,  0.3961,  ...,  0.4118,  0.4039,  0.3961],
         [ 0.6157,  0.4745,  0.4275,  ...,  0.4353,  0.4353,  0.4275],
         ...,
         [ 0.6706,  0.5686,  0.4039,  ...,  0.1686,  0.4980,  0.5059],
         [ 0.6863,  0.5608,  0.3804,  ...,  0.1765,  0.4902,  0.4745],
         [ 0.6941,  0.5686,  0.3804,  ...,  0.1765,  0.4745,  0.4588]],

        [[ 0.5137,  0.3647,  0.3176,  ...,  0.5294,  0.5216,  0.5137],
         [ 0.5294,  0.3882,  0.3412,  ...,  0.3412,  0.3333,  0.3255],
         [ 0.5608,  0.4196,  0.3647,  ...,  0.3490,  0.3490,  0.3490],
         ...,
         [ 0.6235,  0.5216,  0.3647,  ...,  0.0510,  0.3647,  0.3725],
         [ 0.6392,  0.5137,  0.3412,  ...,  0.0588,  0.3569,  0.3412],
         [ 0.6471,  0.5216,  0.3412,  ...,  0.0588,  0.3412,  0.3255]],

        [[ 0.3255,  0.1765,  0.1137,  ...,  0.2706,  0.2627,  0.2549],
         [ 0

# CLIP 불러오기

* CLIP 모델 로드
* 텍스트 임베딩 생성

In [ ]:
#clip설
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-7y69c02j
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-7y69c02j
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=6095599095aa8ed890491efe798921088d1947c156d2dcda8d7719e6070513ed
  Stored in directory: /tmp/pip-ephem-wheel-cache-4irac571/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch

# CLIP 모델 및 프로세서 로드
model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(model_name).cuda()
clip_processor = CLIPProcessor.from_pretrained(model_name)

# 텍스트 장르 준비
genre_list = genredata["Genre"].unique().tolist()  # 유니크한 장르 리스트 생성

# 리스트 요소가 문자열인지 확인
genre_list = [str(genre) for genre in genre_list]  # 모든 항목을 문자열로 변환

# CLIPProcessor로 텍스트를 인코딩
genre_texts = clip_processor(
    text=genre_list, return_tensors="pt", padding=True
).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
#예측 함수
def predict_genres(image_tensor, genre_texts, threshold=0.3):
    image_input = image_tensor.unsqueeze(0).to("cuda")  # 배치 차원 추가
    image_features = clip_model.get_image_features(image_input)

    text_features = clip_model.get_text_features(**genre_texts)

    # 이미지와 텍스트 임베딩 간 코사인 유사도 계산
    similarity = torch.cosine_similarity(image_features, text_features, dim=1)

    predicted_genres = [
        genre_list[i]
        for i, score in enumerate(similarity)
        if score > threshold
    ]
    return predicted_genres


In [ ]:
#genre->idx mapping
genre_to_idx = {g: i for i, g in enumerate(genre_list)}

def encode_genres(genre_str):
    # genre_str 예: "Crime|Drama|Mystery"
    vector = [0]*len(genre_list)
    for gen in genre_str.split("|"):
        gen = gen.strip()
        if gen in genre_to_idx:
            vector[genre_to_idx[gen]] = 1
    return vector


In [ ]:
X = []
y = []
imdb_ids = []

for idx, imdbId, image_tensor, genre_str in final_data:
    X.append(image_tensor)
    y.append(encode_genres(genre_str))
    imdb_ids.append(imdbId)

import torch

X = torch.stack(X)  # (N, 3, 224, 224)
y = torch.tensor(y, dtype=torch.float32)  # (N, genre_count)

In [ ]:
#학습-테스트 데이터 분할
from sklearn.model_selection import train_test_split

imdb_ids_tensor = torch.tensor([int(imdbId) for imdbId in imdb_ids])

X_train, X_test, y_train, y_test, imdb_train, imdb_test = train_test_split(
    X, y, imdb_ids_tensor, test_size=0.2, random_state=42
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])

Train size: 797
Test size: 200
